In [20]:
from keras.models import load_model
import os
import cv2
import numpy
import tensorflow as tf
from keras.models import model_from_json
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import moviepy.editor
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from librosa.core import istft
from tensorflow import keras
#model = model_from_json(open("fer.json", "r").read())
#model.load_weights('fer.h5') 
#face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')    
Model_Emotion_V = keras.models.load_model('V_E_R\\V_E_R.h5')
#model = model_from_json(open("fer.json", "r").read())
model= load_model('F_E_R\\simple_CNN.530-0.65.hdf5')
face_haar_cascade = cv2.CascadeClassifier('F_E_R\\haarcascade_frontalface_default.xml')    

In [17]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        X = librosa.to_mono(X)
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.core.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [22]:
#DataFlair - Load the data and extract features for each sound file
def load_data(path):
 #   file in glob.glob(path)
    x=[]
    feature=extract_feature(path, mfcc=True, chroma=True, mel=True)
    feature = np.reshape(feature,(6,30))
    feature=feature.reshape(1, feature.shape[0], feature.shape[1], 1)
 
    predictions = Model_Emotion_V.predict(feature)
    max_index = numpy.argmax(predictions[0])
    emotion_detection = ('angry','fear','happy','neutral','sad')
    emotion_prediction = emotion_detection[max_index]
    return emotion_prediction

In [27]:
def E_F_R(path):
    
    video =  moviepy.editor.VideoFileClip(path)
    video.audio.write_audiofile(r"output.wav")
    
    
    cap = cv2.VideoCapture(path)
    hap=nat=ang=far=sad=0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
                break

            #Change the frame to greyscale  
        gray_image= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #We pass the image, scaleFactor and minneighbour
        faces_detected = face_haar_cascade.detectMultiScale(gray_image,1.32,5)

            #Draw Triangles around the faces detected
        for (x,y,w,h) in faces_detected:
            cv2.rectangle(frame,(x,y), (x+w,y+h), (255,0,0), thickness=7)
            roi_gray=gray_image[y:y+w,x:x+h]
            roi_gray=cv2.resize(roi_gray,(48,48))

                #Processes the image and adjust it to pass it to the model
            image_pixels = tf.keras.preprocessing.image.img_to_array(roi_gray)
              #  plt.imshow(image_pixels)
               # plt.show()
            image_pixels = np.expand_dims(image_pixels, axis = 0)
            image_pixels /= 255

                #Get the prediction of the model
            predictions = model.predict(image_pixels)
         #   print(predictions)
            max_index = np.argmax(predictions[0])
            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]
            if emotion_prediction=="happy" :
                hap=hap+1
            if emotion_prediction=="angry" :
                ang=ang+1
            if emotion_prediction=="fear" :
                far=far+1
            if emotion_prediction=="sad" :
                sad=sad+1
            if emotion_prediction=="neutral" :
                nat=nat+1

                #Write on the frame the emotion detected
    #        cv2.putText(frame,emotion_prediction,(int(x), int(y)),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)


    #        resize_image = cv2.resize(frame, (700, 700))
    #        cv2.imshow('Emotion',resize_image)
            if cv2.waitKey(10) == ord('b'):
                    break

    cap.release()
    cv2.destroyAllWindows
    

    emotion_detection = ('angry', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    a=[ang,far,hap,sad,nat]
    k=emotion_detection[np.argmax(a)]
    Index=np.argmax(a)
    path="output.wav"
    V_E=load_data(path)
    os.remove("output.wav")
    
    if k!=V_E:
        M_E=k
        a[Index]=0
        k=emotion_detection[np.argmax(a)]
        if k!=V_E:
            k=M_E

    
        
    
    return k

In [28]:
p='C:\\Users\\abd alrhman\\Downloads\\٢٠٢١٠٧٢٤_١٨٥٧٣٧.mp4'

E_F_R(p)

chunk:   0%|                                                                         | 0/180 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in output.wav


MoviePy - Done.


'happy'